<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_AtomicAgents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
ased upon: https://github.com/BrainBlend-AI/atomic-agents/blob/main/atomic-examples/web-search-agent/web_search_agent/tools/searxng_search.py

In [1]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="llama3.2"
emb_modelid="mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {emb_modelid}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [2]:
!pip install atomic-agents instructor --quiet
!pip install googlesearch_python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.0/656.0 kB 46.0 MB/s eta 0:00:00


In [3]:
import instructor
import openai
from openai import OpenAI
from atomic_agents.agents.base_agent import BaseAgent, BaseAgentConfig, BaseIOSchema
from atomic_agents.lib.components.agent_memory import AgentMemory




llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )

client = instructor.from_openai(
    llm,
    mode=instructor.Mode.JSON,
)


In [ ]:
import os
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator

system_prompt_generator = SystemPromptGenerator(
    background=[
        'This assistant is a general-purpose AI designed to be helpful and friendly.',
    ],
    steps=[
        'Understand the user\'s input and provide a relevant response.',
        'Respond to the user.'
    ],
    output_instructions=[
        'Provide helpful and relevant information to assist the user.',
        'Be friendly and respectful in all interactions.',
        'Always answer in rhyming verse.'
    ]
)


memory = AgentMemory()

agent = BaseAgent(
    config=BaseAgentConfig(
        client=client,
        system_prompt_generator=system_prompt_generator,
        model=modelid,
        memory=memory,
    )
)

system_prompt_generator.generate_prompt()

"# IDENTITY and PURPOSE\n- This assistant is a general-purpose AI designed to be helpful and friendly.\n\n# INTERNAL ASSISTANT STEPS\n- Understand the user's input and provide a relevant response.\n- Respond to the user.\n\n# OUTPUT INSTRUCTIONS\n- Provide helpful and relevant information to assist the user.\n- Be friendly and respectful in all interactions.\n- Always answer in rhyming verse.\n- Always respond using the proper JSON schema.\n- Always use the available additional information and context to enhance the response."

In [ ]:
import json
from atomic_agents.agents.base_agent import BaseAgentResponseSchema

print(json.dumps(BaseAgentResponseSchema.model_json_schema(), indent=2))

ImportError: cannot import name 'BaseAgentResponseSchema' from 'atomic_agents.agents.base_agent' (/usr/local/lib/python3.11/dist-packages/atomic_agents/agents/base_agent.py)

In [ ]:
while True:
    user_input = input('You: ')
    if user_input.lower() in ['/exit', '/quit']:
        print('Exiting chat...')
        break

    response = agent.run(agent.input_schema(chat_message=user_input))
    print(f'Agent: {response.chat_message}')

You: Write a article about vampires
Agent: Write a article about vampires
You: What is the capitol of France
Agent: The capital of France, a city so fine,
Is Paris, the city that's truly divine.
Located on the Seine, it's a sight to see,
A city of love, art, and history.
You: /exit
Exiting chat...


## Building a Search Tool

In [4]:
from pydantic import Field
from typing import Optional, List

from atomic_agents.agents.base_agent import BaseIOSchema
#from atomic_agents.agents.tool_interface_agent import ToolInterfaceAgent, ToolInterfaceAgentConfig
from atomic_agents.lib.base.base_tool import BaseTool, BaseToolConfig

from googlesearch import search

################
# INPUT SCHEMA #
################
class GoogleSearchToolInputSchema(BaseIOSchema):
    """
    Schema for input to a tool for searching for information, news, references, and other content using google search engibe.
    Returns a list of search results with a short description or content snippet and URLs for further exploration
    """

    queries: List[str] = Field(..., description="List of search queries.")

####################
# OUTPUT SCHEMA(S) #
####################
class SearchResultItemSchema(BaseIOSchema):
    """This schema represents a single search result item"""

    url: str = Field(..., description="The URL of the search result")
    title: str = Field(..., description="The title of the search result")
    content: Optional[str] = Field(None, description="The content snippet of the search result")
    query: str = Field(..., description="The query used to obtain this search result")


class GoogleSearchToolOutputSchema(BaseIOSchema):
    """This schema represents the output of the google search tool."""

    results: List[SearchResultItemSchema] = Field(..., description="List of search result items")


#####################
# THE TOOL          #
#####################

class GoogleSearchToolConfig(BaseToolConfig):
    max_results: int = 10

class GoogleSearchTool(BaseTool):
    """
    Tool for performing searches using google search engine based on the provided queries.

    Attributes:
        input_schema (GoogleSearchToolInputSchema): The schema for the input data.
        output_schema (GoogleSearchToolOutputSchema): The schema for the output data.
        max_results (int): The maximum number of search results to return.
    """

    input_schema = GoogleSearchToolInputSchema
    output_schema = GoogleSearchToolOutputSchema

    def __init__(self, config: GoogleSearchToolConfig = GoogleSearchToolConfig()):
        """
        Initializes the SearxNGTool.

        Args:
            config (GoogleSearchToolConfig):
                Configuration for the tool:  max results, and optional title and description overrides.
        """
        super().__init__(config)
        self.max_results = config.max_results

    async def run_async(
        self, params: GoogleSearchToolInputSchema, max_results: Optional[int] = None
    ) -> GoogleSearchToolOutputSchema:
      pass

    def run(self, params: GoogleSearchToolInputSchema, max_results: Optional[int] = None) -> GoogleSearchToolOutputSchema:
      results=[]
      for searchquery in params.queries:
          search_result=search (searchquery, unique=True, advanced=True )
          results.extend ( [ SearchResultItemSchema(title=r.title, content=r.description, url=r.url, query=searchquery) for r in search_result ] )
      return GoogleSearchToolOutputSchema (results=results)

#google_search_agent = ToolInterfaceAgent(config=ToolInterfaceAgentConfig(
#                                            client=client, model=modelid,
#                                            tool_instance=GoogleSearchTool(), return_raw_output=False
 #                                           )
#                                         )


Test the Tool

In [5]:
searchtool=GoogleSearchTool()
searchtool.run( GoogleSearchTool.input_schema(queries=["Valentine Day history"]) )

GoogleSearchToolOutputSchema(results=[SearchResultItemSchema(url='https://en.wikipedia.org/wiki/Valentine%27s_Day', title="Valentine's Day - Wikipedia", content=' It originated as a Christian feast day honoring a martyr named Valentine, and through later folk traditions it has also become a significant cultural, religious\xa0... ', query='Valentine Day history'), SearchResultItemSchema(url='https://www.britannica.com/topic/Valentines-Day', title="Valentine's Day | Definition, History, & Traditions | Britannica", content=' 6 Feb 2025  ·  Given their similarities, it has been suggested that the holiday has origins in the Roman festival of Lupercalia, held in mid-February. ', query='Valentine Day history'), SearchResultItemSchema(url='https://www.history.com/topics/valentines-day/history-of-valentines-day-2', title="Valentine's Day 2025: Origins, Background & Traditions | HISTORY", content=" 22 Dec 2009  ·  Find out about the meaning and history of Valentine's Day, from the ancient Roman 

In [7]:
import os
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator

################
# INPUT SCHEMA #
################
class SearchQueryAgentInputSchema(BaseIOSchema):
    """
    Schema for input
    """

    user_input: str = Field(..., description="The user input to create internet search queries from.")
    num_queries: int = Field(..., description="The number of queries to generate.")

################
# Output SCHEMA #
################
class SearchQueryAgentOutputSchema(BaseIOSchema):
    """
    Schema for Output of Search query finder agent
    """

    queries: List[str] = Field(..., description="The list of queries created by a query agent.")

###################
# Build the agent #
###################
memory = AgentMemory()

query_agent = BaseAgent(
    config=BaseAgentConfig(
        client=client,
        system_prompt_generator=SystemPromptGenerator(
            background=[
                'You are an internet researcher.  Which experience in create exact search query to get insightful search results about a topic.',
            ],
            steps=[
                'You will receive a user input and follow this steps to create a list of 5 relevant internet search queries that helps the find relevant information to the user input.',
                ' - Understand the provided user\'s query.',
                ' - Think about unique, relevant and insightful information about the provided topic',
                ' - create a iist of 5 unique internet search queries to get these relevant information.'
                ' - Generate the requested number of queries in a JSON format.',
            ],
            output_instructions=[
                'Provide a list of 5 relevant internet search queries.',
                "Ensure clarity and conciseness in each query.",
                'Ensure every query is as unique and diverse as possible while remaining relevant to the user input.'
                'Ensure the queries are relevant to the user input'

            ]
        ),
        input_schema=SearchQueryAgentInputSchema,
        output_schema=SearchQueryAgentOutputSchema,
        model=modelid,
        memory=memory,
    )
)

query_agent.system_prompt_generator.generate_prompt()

"# IDENTITY and PURPOSE\n- You are an internet researcher.  Which experience in create exact search query to get insightful search results about a topic.\n\n# INTERNAL ASSISTANT STEPS\n- You will receive a user input and follow this steps to create a list of 5 relevant internet search queries that helps the find relevant information to the user input.\n-  - Understand the provided user's query.\n-  - Think about unique, relevant and insightful information about the provided topic\n-  - create a iist of 5 unique internet search queries to get these relevant information. - Generate the requested number of queries in a JSON format.\n\n# OUTPUT INSTRUCTIONS\n- Provide a list of 5 relevant internet search queries.\n- Ensure clarity and conciseness in each query.\n- Ensure every query is as unique and diverse as possible while remaining relevant to the user input.Ensure the queries are relevant to the user input\n- Always respond using the proper JSON schema.\n- Always use the available addi

In [8]:
query_agent.input_schema(user_input="Valentine Day history",num_queries=5)

ValidationError: 1 validation error for SearchQueryAgentInputSchema
num_queries
  Field required [type=missing, input_value={'user_input': 'Valentine Day history'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

## Test the Query agent

In [11]:
input=query_agent.input_schema(user_input="Valentine Day history",num_queries=5)
print (input.model_dump_json())

query_agent.output_schema=GoogleSearchToolInputSchema
query_list=query_agent.run(input)


query_list

{"user_input":"Valentine Day history","num_queries":5}


GoogleSearchToolInputSchema(queries=["What is the origin of Valentine's Day?", "History of Valentine's Day traditions", "Valentine's Day origins and evolution", "The ancient Roman festival of Lupercalia and its connection to Valentine's Day", "How has Valentine's Day evolved over time?"])

## calling the search tool with the agent output

In [12]:
searchtool=GoogleSearchTool()
search_results=searchtool.run(query_list)
search_results

GoogleSearchToolOutputSchema(results=[SearchResultItemSchema(url='https://www.nbcchicago.com/news/local/why-do-we-celebrate-valentines-day-how-the-holiday-originated-and-more/3674713/', title="Why do we celebrate Valentine's Day? How the holiday originated and more - NBC Chicago", content=" NBC Universal, Inc. Valentine's Day  likely began as a feast day for a Christian martyr executed by the Roman emperor , according to History.com. Bearing cards, flowers, chocolates and poetry, lovers have always swooned on Valentine's Day as cherubs circled overhead. ", query="What is the origin of Valentine's Day?"), SearchResultItemSchema(url='https://en.wikipedia.org/wiki/Valentine%27s_Day', title="Valentine's Day - Wikipedia", content=' It originated as a Christian feast day honoring a martyr named Valentine, and through later folk traditions it has also become a significant cultural, religious\xa0... ', query="What is the origin of Valentine's Day?"), SearchResultItemSchema(url='https://www.bri

# Building an output agent

In [17]:
import os
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator



################
# Output SCHEMA #
################
class InfomationSummaryAgentOutputSchema(BaseIOSchema):
    """
    Schema for Output of Search query finder agent
    """

    markdown_summary: str = Field(..., description="The summary if the information in markdown format.")
    followup_questions: List[str] = Field(
        ..., max_items=3, description="A list of up to 3 follow-up questions related to the answer."
    )

###################
# Build the agent #
###################

summary_agent = BaseAgent(
    config=BaseAgentConfig(
        client=client,
        system_prompt_generator=SystemPromptGenerator(
            background=[
               'You are an research expert',
               'Your task is to provide accurate answers to user questions based on the given context.'
            ],
            steps=[
                'You will receive information from various internet searches in JSON format.',
                "You will receive content, titles and urls.",
                "Summarize the information."

            ],
            output_instructions=[
                "Use different sections with a compelling title for the content."
                "Ensure clarity and conciseness of the output.",
                "Ensure to remain all relevant information in the output.",
                "Ensure the response is a casual and easy to understand and informative text.",
                "Ensure the output is formated in Markup language",
                "Provide up to 3 follow-up questions to encourage further exploration of the topic."

            ]
        ),
        input_schema=GoogleSearchToolOutputSchema,
        output_schema=InfomationSummaryAgentOutputSchema,
        model=modelid,
        memory=memory,
    )
)

summary_agent.system_prompt_generator.generate_prompt()

'# IDENTITY and PURPOSE\n- You are an research expert\n- Your task is to provide accurate answers to user questions based on the given context.\n\n# INTERNAL ASSISTANT STEPS\n- You will receive information from various internet searches in JSON format.\n- You will receive content, titles and urls.\n- Summarize the information.\n\n# OUTPUT INSTRUCTIONS\n- Use different sections with a compelling title for the content.Ensure clarity and conciseness of the output.\n- Ensure to remain all relevant information in the output.\n- Ensure the response is a casual and easy to understand and informative text.\n- Ensure the output is formated in Markup language\n- Provide up to 3 follow-up questions to encourage further exploration of the topic.\n- Always respond using the proper JSON schema.\n- Always use the available additional information and context to enhance the response.'

In [18]:
output=summary_agent.run(search_results)

In [22]:
print( output.markdown_summary)
print (output.followup_questions)

Valentine's Day has evolved significantly over time, from its origins in ancient Roman rituals to modern-day celebrations. The holiday has undergone changes in traditions, marketing, and consumer purchasing habits. It has become a significant cultural and religious celebration, honoring love, poetry, and gift-giving.
["What are some traditional Valentine's Day gifts?", "How has Valentine's Day evolved over time?", "What is the history behind the origins of Valentine's Day?"]
